<a href="https://colab.research.google.com/github/shreyasrajesh0308/Deep_Learning/blob/master/Deep_Learning_Book_Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3 - Introduction to Keras and Tensorflow

In [1]:
  import tensorflow as tf

## Constant Tensors and Variables

Define a tensor with shape 2,1 and filled with ones throught 

In [2]:
x = tf.ones(shape=(2, 1))

In [3]:
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


Define a tensor with shape 2,1 filled with zeros

In [4]:
x = tf.zeros(shape=(2,1))

In [5]:
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


Define a tensor of shape 3,1 where elements are picked from a random distribution with mean 0 and standard deviation 1

In [6]:
x = tf.random.normal(shape=(3,1), mean = 0., stddev=1.)

In [7]:
print(x)

tf.Tensor(
[[-0.10958096]
 [-0.43792692]
 [ 1.5117426 ]], shape=(3, 1), dtype=float32)


Define a tensor of shape 3,1 where elements are picked from a uniform distribution with minval 0 and maxval 1

In [8]:
x = tf.random.uniform(shape=(3,1), minval = 0, maxval =1)

In [9]:
x

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.9360405 ],
       [0.23332965],
       [0.7320099 ]], dtype=float32)>

But one caveat with tensors is that their values are not assignable that is particular values cannot be changed for example 

In [10]:
x[0][0] = 10

TypeError: ignored

 But since weights in our network are stored as tensors, how can these update if they cannot be assigned to a value, to do this we have the variable class, which makes these tensors a variable

In [ ]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3,1)))

In [ ]:
v

In [ ]:
v.assign(tf.ones(shape=(3,1)))

In [ ]:
print(v)

In [ ]:
v[0, 0].assign(100)

In [ ]:
v.assign_add(tf.ones(shape=(3,1)))
print(v)

Performing some math operations in tensorflow

In [ ]:
a = tf.ones(shape=(2,2))
b = tf.sqrt(a)
c = tf.sqrt(a)
d = b+c
e = tf.matmul(a, b)
e*=d

In [ ]:
e

Now, we look at the GradientTape API, that is computing the gradient of any differntiable expression